Input: Camflow Logs parsed by unicorn parser

Output Vectorized Graphs

In [1]:
import os 
from os import path as osp
import torch
from tqdm import tqdm, trange
from torch_geometric.data import *
import psycopg2
from psycopg2 import extras as ex

/home/aabouelk/anaconda3/envs/kairos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
connect = psycopg2.connect(
    database = 'camflow',
    host = '/var/run/postgresql/',
    user = 'postgres',
    password = 'postgres',
    port = '5432'
)

cur = connect.cursor()
connect.rollback()

In [3]:
# If the data is already in the database
process_raw_data = True
clean_database = False
node_types = set()
edge_types = set()

In [4]:
if clean_database:    
    for graph_id in trange(220):
        sql = f"delete from raw_data where graph_id='{graph_id}';"
        cur.execute(sql)
        connect.commit()

In [5]:

ds_path = '/home/aabouelk/ds/camflow/'
benign_date = '30-04-2024'
n_splits = 33
graphs_per_file = 3

node_size = []
for idx in trange(n_splits):
    path = osp.join(ds_path, benign_date, f'preprocessed/preprocessed-{idx}.txt')  # The paths to the dataset.
    datalist = []
    lines_in_file = 0
    with open(path, 'rb') as f:
        lines_in_file = sum(1 for _ in f)
    lines_per_graph = lines_in_file / graphs_per_file
    print(lines_per_graph)
    line_count = 0
    graph_idx = 0 
    node_map = {}
    with open(path) as f:
        for line in tqdm(f):
            src_id, dest_id, types = line.strip('\n').split('\t')
            src_type, dest_type, edge_type, ts = types.split(":")
            if src_id not in node_map:
                node_map[src_id] = len(node_map)
            src_id = node_map[src_id]
            if dest_id not in node_map:
                node_map[dest_id] = len(node_map)
            dest_id = node_map[dest_id]
            
            spl = [
                src_id,
                src_type,
                dest_id,
                dest_type,
                edge_type,
                ts,
                (idx * graphs_per_file)+ graph_idx,
                ]
            datalist.append(spl)
            node_types.add(src_type)
            node_types.add(dest_type)
            edge_types.add(edge_type)
            line_count += 1
            if line_count > graph_idx * lines_per_graph:
                graph_idx += 1
                #node_size.append(len(node_map))
                node_map = {}
            if len(datalist) >= 10000 and process_raw_data:
                sql = '''insert into raw_data
                values %s
                '''
                ex.execute_values(cur, sql, datalist, page_size=10000)
                connect.commit()
                datalist = []
# print(node_size)
# print(len(node_map))

  0%|          | 0/33 [00:00<?, ?it/s]

291283.3333333333


873850it [00:24, 35096.95it/s]
  3%|▎         | 1/33 [00:25<13:21, 25.04s/it]

295746.0


887238it [00:24, 36327.20it/s]
  6%|▌         | 2/33 [00:49<12:47, 24.76s/it]

...

891419it [00:25, 35375.39it/s]
 94%|█████████▍| 31/33 [13:21<00:51, 25.73s/it]

296628.6666666667


889886it [00:22, 38722.45it/s]
 97%|█████████▋| 32/33 [13:44<00:24, 24.95s/it]

296836.0


890508it [00:24, 36915.16it/s]
100%|██████████| 33/33 [14:08<00:00, 25.72s/it]


In [6]:
print(len(edge_types))

54


In [50]:
ds_path = '/home/aabouelk/ds/camflow/'
attack_date = '26-05-2024'
a_splits = 1

for idx in trange(10, 14):
    path = osp.join(ds_path, attack_date, f'preprocessed/preprocessed-{idx}.txt')  # The paths to the dataset.
    datalist = []

    graphs_per_file = 25
    lines_in_file = 0
    with open(path, 'rb') as f:
        lines_in_file = sum(1 for _ in f)
    lines_per_graph = lines_in_file / graphs_per_file
    print(lines_per_graph)
    line_count = 0
    graph_idx = 1
    attack_start_idx = 99
    with open(path) as f:
        node_map = {}
        for line in tqdm(f):
            src_id, dest_id, types = line.strip('\n').split('\t')
            src_type, dest_type, edge_type, ts = types.split(":")
            if src_id not in node_map:
                node_map[src_id] = len(node_map)
            src_id = node_map[src_id]
            if dest_id not in node_map:
                node_map[dest_id] = len(node_map)
            dest_id = node_map[dest_id]
            spl = [
                src_id,
                src_type,
                dest_id,
                dest_type,
                edge_type,
                ts,
                attack_start_idx + ((idx-10) * graphs_per_file)+ graph_idx,
                ]
            line_count += 1
            if line_count > graph_idx * lines_per_graph:
                graph_idx += 1
            datalist.append(spl)
            node_types.add(src_type)
            node_types.add(dest_type)
            edge_types.add(edge_type)
            if len(datalist) >= 10000 and process_raw_data:
                sql = '''insert into raw_data
                values %s
                '''
                ex.execute_values(cur, sql, datalist, page_size=10000)
                connect.commit()
                datalist = []

  0%|          | 0/4 [00:00<?, ?it/s]

262225.88


6555647it [02:48, 39012.49it/s]
 25%|██▌       | 1/4 [02:48<08:26, 168.76s/it]

262146.28


6553657it [02:48, 38921.90it/s]
 50%|█████     | 2/4 [05:38<05:38, 169.05s/it]

261996.96


6549924it [02:45, 39472.86it/s]
 75%|███████▌  | 3/4 [08:24<02:48, 168.02s/it]

261927.36


6548184it [02:45, 39518.27it/s]
100%|██████████| 4/4 [11:11<00:00, 167.83s/it]


In [51]:
nodevec = torch.nn.functional.one_hot(torch.arange(0, len(node_types)), num_classes=len(node_types))
edgevec = torch.nn.functional.one_hot(torch.arange(0, len(edge_types)), num_classes=len(edge_types))

edge2onehot = {}
node2onehot = {}
c = 0
for i in node_types:
    node2onehot[i] = nodevec[c]
    c += 1
c = 0
for i in edge_types:
    edge2onehot[i] = edgevec[c]
    c += 1


In [52]:
data_dir = "../data/camflow250"
os.system(f"mkdir -p {data_dir}")
for graph_id in trange(199):
    sql = "select * from raw_data where graph_id='{graph_id}' ORDER BY _id;".format(graph_id=graph_id)
    cur.execute(sql)
    rows = cur.fetchall()
    from torch_geometric.data import TemporalData

    dataset = TemporalData()
    src = []
    dst = []
    msg = []
    t = []
    for i in rows:
        src.append(int(i[0]))
        srconehot = node2onehot[i[1]]
        dst.append(int(i[2]))
        destonehot = node2onehot[i[3]]
        edgeonehot = edge2onehot[i[4]]
        msg_t = torch.cat([srconehot, edgeonehot, destonehot], dim=0)
        msg.append(msg_t)
        t.append(int(i[-1]))    # Use logical order of the event to represent the time
    dataset.src = torch.tensor(src)
    dataset.dst = torch.tensor(dst)
    dataset.t = torch.tensor(t)
    dataset.msg = torch.vstack(msg)
    dataset.src = dataset.src.to(torch.long)
    dataset.dst = dataset.dst.to(torch.long)
    dataset.msg = dataset.msg.to(torch.float)
    dataset.t = dataset.t.to(torch.long)
    torch.save(dataset, f"{data_dir}/graph_" + str(graph_id) + ".TemporalData")

print("end")

100%|██████████| 199/199 [19:40<00:00,  5.93s/it]

end
